In [1]:
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Polygon, Point, MultiPoint
import math
import matplotlib.pyplot as plt
import numpy as np
from numpy import exp
from numpy.random import rand, seed
import folium
import random 
pd.options.mode.chained_assignment = None  # default='warn'

# Model inputs

## Get user input

In [2]:
hub_type = 'centralized' # none, centralized, decentralized

material_input = {
    'biobased_percentage': 'semi', # conventional, semi, full
    'circularity_percentage': 50, # 0 - 100% 
    'modularity_percentage': 20, # 0 - 100% 
}

# TRANSPORTATION INFO
transportation_input = {
    'pre-hub_transportation_type': 'road', # road, roadWater, rail
    'electrification_type': 'mixed', # diesel, mixed, electric 
}

## Make pre-determined data

In [319]:
# a10 boundary
a10 = gpd.read_file('data/A10.shp')
a10 = a10.to_crs('EPSG:28992')

# factory distances
# to add data from route planner: 
# https://www.routescanner.com/app/voyages-search
timberFactory_coords = [50.91608687148152, 5.837795943482761]
concreteFactor_coords = [51.472216701413025, 5.737978710264017]
moduleFactory_coords = [51.218521, 9.637029]
data = {
    'material': ['timber', 'concrete', 'modules'], 
    'distFromAms_km': [500, 250, 350], # dummy number 
    'x': [timberFactory_coords[1], concreteFactor_coords[1], moduleFactory_coords[1]],
    'y': [timberFactory_coords[0], concreteFactor_coords[0], moduleFactory_coords[0]]
}
suppliers = pd.DataFrame(data)
suppliers = gpd.GeoDataFrame(suppliers, crs='EPSG:4326',
                             geometry=gpd.points_from_xy(suppliers.x, suppliers.y))
suppliers = suppliers.to_crs('EPSG:28992')
suppliers = suppliers.drop(columns=['x', 'y'])

# construction project locations and materials 
conSites_df = pd.read_csv('data/constructionSites.csv')
conSites_df = conSites_df.dropna(how='all').dropna(axis=1)
conSites_df.rename(columns={
    'Project name': 'name', 
    'Latitude': 'lat', 
    'Longitude': 'lon', 
    'Developer': 'developer', 
    'Status': 'status', 
    'Type': 'material'
}, inplace=True)

conSites_gdf = gpd.GeoDataFrame(
    conSites_df, 
    geometry=gpd.points_from_xy(conSites_df.lon, conSites_df.lat), 
    crs='EPSG:4326')
conSites_gdf = conSites_gdf.to_crs('EPSG:28992')
conSites_gdf = conSites_gdf[['name', 'geometry']]
conSites = conSites_gdf.copy() #.sample(20)

buildingType_list = ['A', 'B', 'C', 'D', 'E']
conSites['buildingType'] = np.random.choice(buildingType_list, size=len(conSites))
conSites['inA10'] = conSites.geometry.within(a10.geometry[0])

# hub locations for three scenarios: none, centralized, decentralized
minx, miny, maxx, maxy = conSites_gdf.total_bounds
candiHubs = gpd.read_file('data/candiHubs_ams.shp')
candiHubs = candiHubs.cx[minx:maxx, miny:maxy]
microHubs = candiHubs.sample(10)
microHubs['hub_type'] = 'micro'
macroHubs = pd.read_csv('data/data_construction_hubs.csv')
macroHubs = gpd.GeoDataFrame(
    macroHubs, 
    geometry=gpd.points_from_xy(macroHubs.Longitude, macroHubs.Latitude),
    crs='EPSG:4326'
)
macroHubs.rename(columns={'Name': 'name'}, inplace=True)
macroHubs = macroHubs[['name', 'geometry']]
macroHubs = macroHubs.to_crs('EPSG:28992')
macroHubs['hub_type'] = 'macro'
hubs = pd.concat([macroHubs, microHubs]).reset_index(drop=True)
hubs = hubs.reset_index(names='hub_id')
hubs = hubs[['hub_id', 'hub_type', 'geometry']]
macro = hubs[hubs.hub_type == 'macro']
hubs['nearestMacroHub'] = macro.geometry.sindex.nearest(list(hubs.geometry))[1]
hubs['inA10'] = hubs.geometry.within(a10.geometry[0])

# make dummy data for material required per building type 
data_structural = {
    'buildingType': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'D', 'D', 'D', 'E', 'E', 'E'],
    'biobased': ['none', 'semi', 'full', 'none', 'semi', 'full', 'none', 'semi', 'full', 'none', 'semi', 'full', 'none', 'semi', 'full'],
    'concrete': [5000, 3000, 1000, 5000, 3000, 1000, 5000, 3000, 1000, 5000, 3000, 1000, 5000, 3000, 1000],
    'timber': [0, 2000, 4000, 0, 2000, 4000, 0, 2000, 4000, 0, 2000, 4000, 0, 2000, 4000],
}

data_nonStructural = {
    'buildingType': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'D', 'D', 'D', 'E', 'E', 'E'],
    'biobased': ['none', 'semi', 'full', 'none', 'semi', 'full', 'none', 'semi', 'full', 'none', 'semi', 'full', 'none', 'semi', 'full'],
    'concrete': [50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50],
    'timber': [50, 400, 600, 50, 400, 600, 50, 400, 600, 50, 400, 600, 50, 400, 600],
}
data_nUnits = {
    'buildingType': ['A', 'B', 'C', 'D', 'E'],
    'nUnits': [47, 46, 54, 52, 52]
}
nUnits = pd.DataFrame(data_nUnits)
materialPerBiobasedType_structural = pd.DataFrame(data_structural)
materialPerBiobasedType_structural['materialType'] = 'structural'
materialPerBiobasedType_nonstructural = pd.DataFrame(data_nonStructural)
materialPerBiobasedType_nonstructural['materialType'] = 'nonstructural'
material_perBiobasedType = pd.concat([materialPerBiobasedType_structural, materialPerBiobasedType_nonstructural])
for mat in ['concrete', 'timber']: 
    material_perBiobasedType[mat] = material_perBiobasedType[mat].map(lambda x: x+random.randint(50,500))
material_perBiobasedType = pd.merge(material_perBiobasedType, nUnits)
    
# vehicle capacities
data = {
    'transType': ['road', 'road', 'roadWater', 'rail'],
    'vehicleType': ['diesel', 'electric', 'roadWater', 'rail'],
    'capacity_timber': [20, 20, 50, 45],
    'capacity_concrete': [25, 25, 60, 70],
    'capacity_modules': [2, 2, 3, 5],
    'emissions_perKm': [0.0009, 0, 0.0007, 0.0006]
}
vehicleCapacity_df = pd.DataFrame(data)

# to add: demolition site locations 
# to add: street network

# Define functions

In [388]:
class ConstructionSites: 
    def __init__(self, gdf, biobased_type, circular_percentage, modularity_type): 
        self.biobased_type = biobased_type
        self.circular_percentage = circular_percentage
        self.modularity_type = modularity_type
        self.gdf = gdf
        
    def calc_materialsRequired(self, material_perBiobasedType): 
        biobased_type = self.biobased_type
        conSites = self.gdf
        
        def matRequired(row): 
            buildingType = row.buildingType
            matList = ['concrete', 'timber']
            for mat in matList: 
                mat_df = material_perBiobasedType.copy()
                mat_df = mat_df[(mat_df.biobased == biobased_type) & 
                                (mat_df.buildingType == buildingType)] 
                row[f'total_{mat}_tons'] = mat_df[mat].sum()
                for struc_type in ['structural', 'nonstructural']: 
                    mat_df_t = mat_df[mat_df.materialType == struc_type]
                    row[f'{struc_type}_{mat}_tons'] = mat_df_t[mat].iloc[0]
            row['nUnits'] = mat_df.nUnits.iloc[0]
            return row
        conSites = conSites.apply(lambda row: matRequired(row), axis=1)
        self.gdf = conSites
    
class Hubs: 
    def __init__(self, gdf, hub_network):
        self.hub_network = hub_network
        self.gdf = gdf
        
class Model(ConstructionSites, Hubs): 
    def __init__(self):
        None
        
    def load_data(self, constructionSites_gdf, hubs_gdf, material_perBuildingType_df, 
                  suppliers_df, vehicleCapacity_df): 
        self.constructionSites = constructionSites_gdf
        self.hubs = hubs_gdf
        self.material_perBuildingType = material_perBuildingType_df
        self.suppliers = suppliers_df
        self.vehicleCapacities = vehicleCapacity_df
    
    def load_parameters(self, biobased_type, circular_percentage, modularity_type, 
                        hub_network, transportation_network, vehicle_type):
        
        # initialize ConstructionSites and Hubs
        conSites = ConstructionSites(self.constructionSites, biobased_type, circular_percentage, modularity_type)
        conSites.calc_materialsRequired(self.material_perBuildingType)
        self.constructionSites = conSites.gdf
        self.hubs = Hubs(self.hubs, hub_network).gdf
        
        # add parameters
        self.circular_percentage = circular_percentage
        self.modularity_type = modularity_type
        self.transportation_network = transportation_network
        self.vehicle_type = vehicle_type
        self.hub_network = hub_network
    
    def run_model(self): 
        self.find_nearestHubs()
        self.calc_distances()
        self.constructionSites = self.calc_nTrucks(self.constructionSites)
        self.calc_emissions_h2c()
        self.pickHubs_s2h()
        self.hubs = self.calc_nTrucks(self.hubs)
        self.calc_emissions_s2h()
        self.calc_emissions_total()
        
    def find_nearestHubs(self): 
        conSites = self.constructionSites
        hubs = self.hubs 
        macroHubs = hubs[hubs.hub_type == 'macro']
        conSites['nearestHub'] = hubs.geometry.sindex.nearest(list(conSites.geometry))[1]
        conSites['nearestMacroHub'] = macroHubs.geometry.sindex.nearest(list(conSites.geometry))[1]
        self.constructionSites = conSites
        
    def calc_distances(self): 
        conSites = self.constructionSites
        hubs = self.hubs 
        
        def calc_dists_cons2hub(row): 
            macroHubs = hubs[hubs.hub_type == 'macro']
            conGeom = row.geometry
            hubGeom = hubs[hubs.hub_id == row.nearestHub].geometry.iloc[0]
            hubGeom_macro = macroHubs[macroHubs.hub_id == row.nearestMacroHub].geometry.iloc[0]
            row['dist_micro2cons'] = conGeom.distance(hubGeom)
            row['dist_macro2cons'] = conGeom.distance(hubGeom_macro)
            row['dist_macro2micro'] = hubGeom_macro.distance(hubGeom)
            return row 
        conSites = conSites.apply(lambda row: calc_dists_cons2hub(row), axis=1)
        self.constructionSites = conSites
        
    def calc_nTrucks_h2c(self): 
        capacities = self.vehicleCapacities
        capacity = capacities[
            (capacities.transType == 'road') & 
            (capacities.vehicleType == self.vehicle_type)
        ]
        capacity_modular = capacity[f'capacity_modules'].iloc[0]
        
        gdf = self.constructionSites
        def calc_nTrucks_lambda(row, modularity): 
            nTrucks_modular_list = []
            for mat in ['concrete', 'timber']: 
                mat_nonStructural = row[f'nonstructural_{mat}_tons']
                mat_structural = row[f'structural_{mat}_tons']
                capacity_notModular = capacity[f'capacity_{mat}'].iloc[0]
                if modularity == 'full': 
                    row[f'nTrucks_{mat}'] = math.ceil(mat_structural / capacity_notModular)
                    nTrucks_modular = math.ceil(row.nUnits / capacity_modular)
                    nTrucks_modular_list.append(nTrucks_modular)
                elif modularity == 'none':
                    row[f'nTrucks_{mat}'] = math.ceil((mat_nonStructural + mat_structural) / capacity_notModular)
            nTrucks_modular_total = sum(nTrucks_modular_list)
            row['nTrucks_modules'] = nTrucks_modular_total
            return row 
        gdf = gdf.apply(lambda row: calc_nTrucks_lambda(row, self.modularity_type), axis=1)
        self.constructionSites = gdf
            
    def calc_emissions_h2c(self): 
        conSites = self.constructionSites
        hubs = self.hubs
        vehicle_type = self.vehicle_type
        hub_network = self.hub_network 
        
        emissionsPerKm_tons_diesel = 0.0009 
        emissionsPerKm_tons_electric = 0

        def calc_emissions_h2c_lambda(row):            
            conSite_inA10 = row.inA10
            hub_inA10 = hubs[hubs.hub_id == row.nearestHub].inA10.iloc[0]
            nTrucks_list = []
            for mat in ['timber', 'concrete'] + ['modules']: 
                nTrucks_list.append(row[f'nTrucks_{mat}'])
            nTrucks = sum(nTrucks_list)
            
            dist_macro2micro = row.dist_macro2micro / 1000
            dist_micro2cons = row.dist_micro2cons / 1000
            if hub_network == 'centralized': 
                dist_tot = row.dist_macro2cons / 1000
            elif hub_network == 'decentralized': 
                dist_tot = (row.dist_macro2micro + row.dist_micro2cons) / 1000
            else: # hub_network == 'none'
                dist_tot = 0 

            # calc emissions based on conditions
            if vehicle_type == 'diesel': 
                row['emissions_h2c'] = dist_tot * nTrucks * emissionsPerKm_tons_diesel
            elif vehicle_type == 'semi' and hub_network == 'decentralized':
                if hub_inA10: 
                    row['emissions_h2c'] = dist_tot * nTrucks * emissionsPerKm_tons_electric
                else: # hub outside A10  
                    emissions_macro2micro = dist_macro2micro * nTrucks * emissionsPerKm_tons_diesel
                    if conSite_inA10: 
                        emissions_micro2cons = dist_micro2cons * nTrucks * emissionsPerKm_tons_electric
                    else: # consite outside A10 
                        emissions_micro2cons = dist_micro2cons * nTrucks * emissionsPerKm_tons_diesel
                    row['emissions_h2c'] = emissions_macro2micro + emissions_micro2cons
            elif vehicle_type == 'semi' and hub_network == 'centralized':
                if conSite_inA10:
                    row['emissions_h2c'] = dist_tot * nTrucks * emissionsPerKm_tons_electric
                else: # conSite outside A10: 
                    row['emissions_h2c'] = dist_tot * nTrucks * emissionsPerKm_tons_diesel
            elif vehicle_type == 'electric': 
                row['emissions_h2c'] = dist_tot * nTrucks * emissionsPerKm_tons_electric
            if hub_network == 'none': 
                row['emissions_h2c'] = 0 
                
            return row       
        conSites = conSites.apply(lambda row: calc_emissions_h2c_lambda(row), axis=1)
        self.constructionSites = conSites
        self.emissions_h2c = conSites.emissions_h2c.sum()
        
    def pickHubs_s2h(self): 
        conSites = self.constructionSites
        hubs = self.hubs
        hub_network = self.hub_network
        if hub_network == 'none': 
            if self.transportation_network != 'road': 
                print('''if the 'no hubs' scenario is chosen, road network is the only option.''')
            hubs = conSites.copy() 
            hubs.rename(columns={'name': 'hub_id'}, inplace=True)
        else: # hub_network == 'centralized' or 'decentralized'
            matRequired = conSites.groupby(by='nearestMacroHub').sum(numeric_only=True).reset_index()
            matRequired.rename(columns={'nearestMacroHub': 'hub_id'}, inplace=True)
            hubs = hubs.merge(matRequired, on='hub_id')
        self.hubs = hubs

    def pick_vehicleCapacity(self): 
        transType = self.transportation_network
        vehicleType = transType if transType != 'road' else 'diesel'
        capacities = self.vehicleCapacities
        capacity = capacities[
            (capacities.transType == self.transportation_network) & 
            (capacities.vehicleType == vehicleType)
        ]
        self.vehicleCapacity_current = capacity
        
    def calc_nVehicles_s2h(self): 
        '''calc nTrucks according to transNetwork, vehicleType'''
        gdf = self.hubs
        capacity = self.vehicleCapacity_current
        transType = self.transportation_network
        capacity_modular = capacity[f'capacity_modules'].iloc[0]
        def calc_nVehicles_lambda(row, modularity): 
            nVehicles_modular_list = []
            for mat in ['concrete', 'timber']: 
                mat_nonStructural = row[f'nonstructural_{mat}_tons']
                mat_structural = row[f'structural_{mat}_tons']
                capacity_notModular = capacity[f'capacity_{mat}'].iloc[0]
                if modularity == 'full': 
                    row[f'nVehicles_{mat}'] = math.ceil(mat_structural / capacity_notModular)
                    nVehicles_modular = math.ceil(row.nUnits / capacity_modular)
                    nVehicles_modular_list.append(nVehicles_modular)
                elif modularity == 'none':
                    row[f'nVehicles_{mat}'] = math.ceil((mat_nonStructural + mat_structural) / capacity_notModular)
            nVehicles_modular_total = sum(nVehicles_modular_list)
            row['nVehicles_modules'] = nVehicles_modular_total
            return row 
        gdf = gdf.apply(lambda row: calc_nVehicles_lambda(row, self.modularity_type), axis=1)
        self.hubs = gdf 
        
    def calc_emissions_s2h(self): 
        suppliers = self.suppliers
        hubs = self.hubs
        capacity = self.vehicleCapacity_current
        transType = self.transportation_network
        vehicleType = transType if transType != 'road' else 'diesel'
        capacities = self.vehicleCapacities
        capacity = capacities[
            (capacities.transType == self.transportation_network) & 
            (capacities.vehicleType == vehicleType)
        ]
        emissionsPerKm = capacity.emissions_perKm.iloc[0]
                        
        def calc_emissions_s2h_lambda(row): 
            emissions_list = []
            for mat in ['concrete', 'timber'] + ['modules']: 
                nTrucks = row[f'nVehicles_{mat}']
                distance = suppliers[suppliers.material == mat].distFromAms_km.iloc[0]
                emissions = distance * nTrucks * emissionsPerKm
                emissions_list.append(emissions)
            row['emissions_s2h'] = sum(emissions_list) 
            return row 
        hubs = hubs.apply(lambda row: calc_emissions_s2h_lambda(row), axis=1)
        self.emissions_s2h = hubs.emissions_s2h.sum()
        self.hubs = hubs
    
    def calc_emissions_total(self): 
        self.emissions_total = self.emissions_s2h + self.emissions_h2c
        
    def visualize(self, zoom_start=12): 
        self.make_gdfs_forPlotting()
        self.make_coords_forPlotting()
        
        m = folium.Map([52.377231, 4.899288], zoom_start=zoom_start, tiles='cartodbdark_matter')
        self.folium_plotPolygons(m, a10, 'grey')
        if self.hub_network == 'none': 
            folium.PolyLine(locations=self.coords_supplier2site, weight=1, color='#4a4a4a', dash_array='5').add_to(m)
        elif self.hub_network == 'centralized': 
            folium.PolyLine(locations=self.coords_supplier2macro, weight=1, color='#4a4a4a', dash_array='5').add_to(m)
            folium.PolyLine(locations=self.coords_macro2site, weight=1, color='white', dash_array='5').add_to(m)
        elif self.hub_network == 'decentralized': 
            folium.PolyLine(locations=self.coords_supplier2macro, weight=1, color='#4a4a4a', dash_array='5').add_to(m)
            folium.PolyLine(locations=self.coords_macro2micro, weight=1, color='white', dash_array='5').add_to(m)
            folium.PolyLine(locations=self.coords_micro2site, weight=1, color='white', dash_array='5').add_to(m)
        self.folium_plotPoints(m, self.macroHubs_plot, 'red', radius=5)
        self.folium_plotPoints(m, self.microHubs_plot, 'red')
        self.folium_plotPoints(m, self.conSites_plot, 'lightblue')
        return m

    def make_gdfs_forPlotting(self): 
        self.hubs_plot = self.hubs.to_crs('EPSG:4326')
        self.conSites_plot = self.constructionSites[['geometry', 'nearestHub', 'nearestMacroHub']].to_crs('EPSG:4326')
        self.macroHubs_plot = self.hubs_plot[hubs_plot.hub_type == 'macro']
        self.microHubs_plot = self.hubs_plot[hubs_plot.hub_type == 'micro']
        self.suppliers_plot = self.suppliers.to_crs('EPSG:4326')
        
    def make_coords_forPlotting(self): 
        conSites_plot = self.conSites_plot
        hubs_plot = self.hubs_plot
        macroHubs_plot = self.microHubs_plot
        self.coords_supplier2site = self.makeCoords_lines_supplier2dest(conSites_plot)
        self.coords_supplier2macro = self.makeCoords_lines_supplier2dest(macroHubs_plot)
        self.coords_macro2site = self.makeCoords_lines_hub2cons(hub_type='macro')
        self.coords_micro2site = self.makeCoords_lines_hub2cons(hub_type='micro')
        self.coords_macro2micro = self.makeCoords_lines_macro2micro()
        
    def folium_plotPoints(self, m, gdf, color, radius=1):
        gdf = gdf.to_crs('EPSG:4326')
        gdf['x'] = gdf.geometry.x
        gdf['y'] = gdf.geometry.y
        locList = gdf[['y', 'x']].values.tolist()
        for loc in locList:
            folium.CircleMarker(location=loc, radius=radius, fill_color=color, color=color).add_to(m)

    def folium_plotPolygons(self, m, gdf, color): 
        gdf = gdf.to_crs('EPSG:4326')
        coords = list(gdf.geometry[0].exterior.coords)
        coords = [[x[1], x[0]] for x in coords]
        folium.Polygon(locations = coords, color=color,
                       fill_color=color, fill_opacity=0.2, fill=True).add_to(m)
        
    def geom_to_coordsList(self, geometrySeries):
        geometrySeries = gpd.GeoSeries(geometrySeries)
        coords_list = list(zip(list(geometrySeries.y), list(geometrySeries.x)))
        coords_list = [list(x) for x in coords_list]
        return coords_list
    
    def makeCoords_lines_hub2cons(self, hub_type):
        hubs_plot=self.hubs_plot
        conSites_plot=self.conSites_plot
        
        left_on = 'nearestHub' if hub_type == 'micro' else 'nearestMacroHub'
        temp = conSites_plot.merge(hubs_plot, left_on=left_on, right_on='hub_id')
        temp.rename(columns={'geometry_x': 'geom_con', 'geometry_y': 'geom_hub'}, inplace=True)
        temp = temp[['geom_con', 'geom_hub']]

        coord_con = self.geom_to_coordsList(temp.geom_con)
        coord_hub = self.geom_to_coordsList(temp.geom_hub)
        coord_list = list(zip(coord_con, coord_hub))
        coord_list = [list(x) for x in coord_list]
        return coord_list

    def makeCoords_lines_macro2micro(self):
        hubs_plot=self.hubs_plot
        macroHubs_plot = hubs_plot[hubs_plot.hub_type == 'macro']
        microHubs_plot = hubs_plot[hubs_plot.hub_type == 'micro']
        t = microHubs_plot.merge(macroHubs_plot, left_on='nearestMacroHub', right_on='hub_id')
        t.rename(columns={'geometry_x': 'geom_micro', 'geometry_y': 'geom_macro'}, inplace=True)
        t = t[['geom_micro', 'geom_macro']]

        coord_micro = self.geom_to_coordsList(t.geom_micro)
        coord_macro = self.geom_to_coordsList(t.geom_macro)
        coord_list = list(zip(coord_micro, coord_macro))
        coord_list = [list(x) for x in coord_list]
        return coord_list

    def makeCoords_lines_supplier2dest(self, dest_plot):
        suppliers_plot=self.suppliers_plot
        coord_list = []
        for material in ['timber', 'concrete']: 
            supplier_geom = self.geom_to_coordsList(suppliers_plot[suppliers_plot.material==material].geometry)[0]
            supplier_coords = [supplier_geom for i in range(len(dest_plot))]
            dest_coords = self.geom_to_coordsList(dest_plot.geometry)
            coord_list_mat = list(zip(supplier_coords, dest_coords))
            coord_list_mat = [list(x) for x in coord_list_mat]
            coord_list.extend(coord_list_mat)
        return coord_list

In [389]:
biobased_type = 'none' # none, semi, full 
circular_percentage = 50
modularity_type = 'full' # none, full 

model = Model()
model.load_data(conSites, hubs, material_perBiobasedType, suppliers, vehicleCapacity_df)
model.load_parameters(biobased_type='full', circular_percentage=50, modularity_type='none', 
                  hub_network='centralized', transportation_network='rail', vehicle_type='diesel')
# model.run_model()
model.find_nearestHubs()
model.calc_distances()
model.calc_nTrucks_h2c()
model.calc_emissions_h2c()
model.pickHubs_s2h()
model.pick_vehicleCapacity()
model.calc_nVehicles_s2h()
model.calc_emissions_s2h()
# model.calc_emissions_total()

c = model.constructionSites
h = model.hubs
s = model.suppliers

h[['nVehicles_concrete', 'nVehicles_timber', 'nVehicles_modules']]

,nVehicles_concrete,nVehicles_timber,nVehicles_modules
0,336,1531,0
1,323,1654,0


In [386]:
capacities = model.vehicleCapacities
capacity = capacities[capacities.transType == 'roadWater']
capacity.emissions_perKm.iloc[0]

0.0007